In [1]:
import os
import glob
import shutil

import pydicom
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils.data_utils import Sequence
from tensorflow.keras.utils import multi_gpu_model
import mlflow.tensorflow as mlflow_tensorflow

from src.mask_functions import rle2mask
from src.train_utils import *
from src.unet import *
from src.fcn import *

os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1"
os.environ["NO_PROXY"] = "localhost, 0.0.0.0, 127.0.0.1"
os.environ["no_proxy"] = "localhost, 0.0.0.0, 127.0.0.1"
get_available_gpus()

W1005 10:26:06.303832 140408548263680 deprecation_wrapper.py:119] From /home/eoconnell/pneumothorax_segmentation/src/resnet_helpers.py:1: The name tf.keras.layers.CuDNNGRU is deprecated. Please use tf.compat.v1.keras.layers.CuDNNGRU instead.

W1005 10:26:06.307426 140408548263680 deprecation_wrapper.py:119] From /home/eoconnell/pneumothorax_segmentation/src/resnet_helpers.py:1: The name tf.keras.layers.CuDNNLSTM is deprecated. Please use tf.compat.v1.keras.layers.CuDNNLSTM instead.



['/device:GPU:0', '/device:GPU:1']

In [2]:
train_annotations = "/home/eoconnell/datasets/kaggle/pneumothorax_segmentation/train-rle.csv"
train_data_loc = "/home/eoconnell/datasets/kaggle/pneumothorax_segmentation/dicom-images-train/"

img_width = 256
img_height = 256
img_channel = 1
batch_size = 8

file_locs = glob.glob(train_data_loc + "/*/*/*.dcm")
df = pd.read_csv(train_annotations, skipinitialspace=True)

df = df[df["EncodedPixels"] != "-1"]



train_files, test_files = train_test_split(
    file_locs, test_size=0.2, random_state=42)

train_data_gen = data_gen(df, train_files,
                          img_width=img_width,
                          img_height=img_height,
                          img_channel=img_channel, 
                          batch_size=batch_size)
test_data_gen = data_gen(df, test_files,
                         img_width=img_width,
                         img_height=img_height,
                         img_channel=img_channel)

In [12]:
df["ImageId"].values[0]


'1.2.276.0.7230010.3.1.4.8323329.4904.1517875185.355709'

In [4]:
i = 0
for img, mask in tqdm(train_data_gen):
#    img = np.squeeze(img[0])
#    mask = np.squeeze(mask[0])
#        
#    if np.sum(mask) < 1:
#        continue
#    
#    fig, ax = plt.subplots(1, 3)
#    img_w_mask = img.copy()
#    img_w_mask[mask.astype(bool)] = 255
#   
#    fig.set_figwidth(20)
#    fig.set_figheight(10)
#        
#    ax[0].imshow(img, cmap=plt.cm.bone)
#    ax[1].imshow(mask)
#    ax[2].imshow(img_w_mask, cmap=plt.cm.bone)
    break

In [5]:
i = 0
for img, mask in tqdm(train_data_gen):
    print(mask.shape)
    for x in mask:
        print(np.sum(x))
    break

(8, 256, 256, 1)
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0



In [6]:
nb_epoch = 100
lr_base = 0.001
steps_per_epoch=len(train_files)
validation_steps=len(test_files)
multi_gpu = False

def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

mlflow.set_tracking_uri("http://0.0.0.0:6600")
mlflow.set_experiment("pneumothorax_segmentation")

with mlflow.start_run():
    
    # Log hyperparameters
    mlflow_tensorflow.autolog(every_n_iter=1)
    mlflow.log_param("batch size", batch_size)
    mlflow.log_param("epochs", nb_epoch)
    mlflow.log_param("learning rate", lr_base)


    model = unet(input_size=(img_width, img_height, img_channel))
    #model = FCN_32s((img_width, img_height, img_channel))
    model.summary()

    if multi_gpu:
        model = multi_gpu_model(model, gpus=4)

    model.compile(loss=dice_coef_loss,
                  optimizer=tf.train.AdamOptimizer(learning_rate=lr_base),
                  metrics=['binary_crossentropy', dice_coef])

    log_dir = "/home/eoconnell/pneumothorax_segmentation/model_dir/"
    if os.path.isdir(log_dir):
        print("is dir")
        shutil.rmtree(log_dir)


    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir)

    tf.keras.callbacks.ModelCheckpoint(log_dir, monitor='val_loss', save_best_only=True)

    history = model.fit_generator(train_data_gen,
                        validation_data=test_data_gen,
                        steps_per_epoch=steps_per_epoch//batch_size,
                        validation_steps=validation_steps//batch_size,
                        epochs=nb_epoch,
#                        callbacks=[tensorboard_callback],
                        workers=10,
                        use_multiprocessing=True)
    log_history_obj(history)
    mlflow.log_artifacts(log_dir)

W1005 10:26:07.581900 140408548263680 deprecation.py:506] From /home/eoconnell/.local/lib/python3.5/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1005 10:26:07.640537 140408548263680 deprecation.py:323] From /home/eoconnell/.local/lib/python3.5/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 128, 128, 64) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

Process Keras_worker_ForkPoolWorker-1:
Process Keras_worker_ForkPoolWorker-3:
Traceback (most recent call last):
Process Keras_worker_ForkPoolWorker-7:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Process Keras_worker_ForkPoolWorker-2:
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Process Keras_worker_ForkPoolWorker-8:
Traceback (most recent call last):
Traceback (most recent call last):
Process Keras_worker_ForkPoolWorker-4:
Process Keras_worker_ForkPoolWorker-10:
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Process Keras_worker_ForkPoolWorker-9:
Process Keras_worker_ForkPoolWorker-6:
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
  

KeyboardInterrupt: 